# Introduction

My goal with this notebook is to show the direct relation between the paper (Show, Attend and Tell) and it's implementation within this repository. I will try to explain the code as much as possible, nontheless recommend you to read the paper first.

The implementation is based on the [AaronCCWong's implementation](https://github.com/AaronCCWong/Show-Attend-and-Tell). I build on top of his work, trying some alternative techniques (like using pre-trained embeddings) and more closely aligning it with the original paper.

## The Encoder

Section 3.1.1 in the "Show, Attend and Tell" paper describes the encoder. The proposed model uses a CNN to extract a set of feature vectors, referred to as annotation vectors. This code implements that concept by using pre-trained models (VGG19, ResNet152, or DenseNet161), modifying them to exclude the final classification layers, and reshaping the output to form a set of feature vectors (our attention vectors). Each vector represents different spacial parts of the image, which is key for the attention mechanism in the next stages of the model.

```python
import torch.nn as nn
from torchvision.models import densenet161, resnet152, vgg19
from torchvision.models import VGG19_Weights

class Encoder(nn.Module):
    """
    Encoder network for image feature extraction, follows section 3.1.1 of the paper
    """
    def __init__(self, network='vgg19'):
        super(Encoder, self).__init__()
        self.network = network
        # Selection of pre-trained CNNs for feature extraction
        if network == 'resnet152':
            self.net = resnet152(pretrained=True)
            # Removing the final fully connected layers of ResNet152
            self.net = nn.Sequential(*list(self.net.children())[:-2])
            self.dim = 2048  # Dimension of feature vectors for ResNet152
        elif network == 'densenet161':
            self.net = densenet161(pretrained=True)
            # Removing the final layers of DenseNet161
            self.net = nn.Sequential(*list(list(self.net.children())[0])[:-1])
            self.dim = 1920  # Dimension of feature vectors for DenseNet161
        else:
            self.net = vgg19(weights=VGG19_Weights.DEFAULT)
            # Using features from VGG19, excluding the last pooling layer
            self.net = nn.Sequential(*list(self.net.features.children())[:-1])
            self.dim = 512  # Dimension of feature vectors for VGG19

    def forward(self, x):
        x = self.net(x)
        # These steps correspond to the extraction of annotation vectors (a = {a1,...,aL}) as described in Section 3.1.1 of the paper.
        # 1. Change the order from (BS, C, H, W) to (BS, H, W, C) in prep for reshaping
        x = x.permute(0, 2, 3, 1)
        # 2. Reshape to [BS, num_spatial_features, C], the -1 effectively flattens the height and width dimensions into a single dimension
        x = x.view(x.size(0), -1, x.size(-1))
        return x
```


## The Decoder

Let's move forward to Section 3.1.2, explaining the Decoder. The Decoder uses the extracted image features to initialize the states of the LSTM cell (by averaging them) and then employs an **attention mechanism** at each time step to focus on different parts of the image while generating the caption. The Decoder predicts one word of the caption at each time step, and its prediction is conditioned on the current LSTM state, the context vector from the attention mechanism, and the previous word. Teacher forcing, a common technique in training sequence generation models where the ground truth word is fed as the next input instead of the model's prediction, is optionally used.

Provided below is the full implementation of the Decoder. I will break the code down into smaller chunks to explore it in more detail, based on the paper's description of the Decoder.

```python
import torch
import torch.nn as nn
from attention import Attention


class Decoder(nn.Module):
    def __init__(self, vocabulary_size, encoder_dim, tf=False):
        super(Decoder, self).__init__()
        self.use_tf = tf

        # Initializing parameters
        self.vocabulary_size = vocabulary_size
        self.encoder_dim = encoder_dim

        # Initial LSTM cell state generators
        self.init_h = nn.Linear(encoder_dim, 512)  # For hidden state
        self.init_c = nn.Linear(encoder_dim, 512)  # For cell state
        self.tanh = nn.Tanh()

        # Attention mechanism related layers
        self.f_beta = nn.Linear(512, encoder_dim)  # Gating scalar in attention mechanism
        self.sigmoid = nn.Sigmoid()

        # Output layer and embedding
        self.deep_output = nn.Linear(512, vocabulary_size)  # Maps LSTM outputs to vocabulary
        self.dropout = nn.Dropout()

        # Attention and LSTM components
        self.attention = Attention(encoder_dim)  # Attention network
        self.embedding = nn.Embedding(vocabulary_size, 512)  # Embedding layer for input words
        self.lstm = nn.LSTMCell(512 + encoder_dim, 512)  # LSTM cell

    def forward(self, img_features, captions):
        # Forward pass of the decoder
        batch_size = img_features.size(0)

        # Initialize LSTM state
        h, c = self.get_init_lstm_state(img_features)

        # Teacher forcing setup
        max_timespan = max([len(caption) for caption in captions]) - 1
        prev_words = torch.zeros(batch_size, 1).long().to(mps_device)
        if self.use_tf:
            embedding = self.embedding(captions) if self.training else self.embedding(prev_words)
        else:
            embedding = self.embedding(prev_words)

        # Preparing to store predictions and attention weights
        preds = torch.zeros(batch_size, max_timespan, self.vocabulary_size).to(mps_device)
        alphas = torch.zeros(batch_size, max_timespan, img_features.size(1)).to(mps_device)

        # Generating captions
        for t in range(max_timespan):
            context, alpha = self.attention(img_features, h)  # Compute context vector via attention
            gate = self.sigmoid(self.f_beta(h))  # Gating scalar for context
            gated_context = gate * context  # Apply gate to context

            # Prepare LSTM input
            if self.use_tf and self.training:
                lstm_input = torch.cat((embedding[:, t], gated_context), dim=1)
            else:
                embedding = embedding.squeeze(1) if embedding.dim() == 3 else embedding
                lstm_input = torch.cat((embedding, gated_context), dim=1)

            # LSTM forward pass
            h, c = self.lstm(lstm_input, (h, c))
            output = self.deep_output(self.dropout(h))  # Generate word prediction

            preds[:, t] = output
            alphas[:, t] = alpha  # Store attention weights

            # Prepare next input word
            if not self.training or not self.use_tf:
                embedding = self.embedding(output.max(1)[1].reshape(batch_size, 1))
        return preds, alphas

    def get_init_lstm_state(self, img_features):
        # Initializing LSTM state based on image features
        avg_features = img_features.mean(dim=1)

        c = self.init_c(avg_features)  # Cell state
        c = self.tanh(c)

        h = self.init_h(avg_features)  # Hidden state
        h = self.tanh(h)

        return h, c
```

### The deep_output layer

Show, Attend and Tell utilizes a *deep output layer* (Pascanu et al., 2014) to compute the output word probability given given the current state of the LSTM, the context vector from the attention mechanism, and the previously generated word.

Let's break down this formula and map its components to the code:

$$ p\left(\mathbf{y}_t \mid \mathbf{a}, \mathbf{y}_1^{t-1}\right) \propto \exp \left(\mathbf{L}_o\left(\mathbf{E} \mathbf{y}_{t-1}+\mathbf{L}_h \mathbf{h}_t+\mathbf{L}_z \hat{\mathbf{z}}_t\right)\right) $$



Where p of $ \mathbf{y}_t $ is the probability of the output word at time $ t $ given the image features $ \mathbf{a} $ and the previously generated words $ \mathbf{y}_1^{t-1} $.

In this formula:

- $ \mathbf{y}_t $ is the output word at time $ t $.
- $ \mathbf{a} $ represents the set of annotation vectors (image features).
- $ \mathbf{y}_1^{t-1} $ are the previously generated words up to time $ t-1 $.
- $ \mathbf{L}_o, \mathbf{L}_h, \mathbf{L}_z $ are learned weight matrices.
- $ \mathbf{E} $ is the embedding matrix for the previous word $ \mathbf{y}_{t-1} $.
- $ \mathbf{h}_t $ is the hidden state of the LSTM at time $ t $.
- $ \hat{\mathbf{z}}_t $ is the context vector at time $ t $, generated by the attention mechanism.

Now let's map this to the Decoder's code:

1. **Embedding of the Previous Word ($ \mathbf{E} \mathbf{y}_{t-1} $)**: This is done using the `self.embedding` layer in the code.

    ```python
    embedding = self.embedding(prev_words)
    ```

2. **Hidden State of the LSTM ($ \mathbf{h}_t $)**: The `h` variable in the code represents the hidden state of the LSTM at each time step.

    ```python
    h, c = self.lstm(lstm_input, (h, c))
    ```

3. **Context Vector ($ \hat{\mathbf{z}}_t $)**: The context vector is computed by the attention mechanism in the `self.attention` layer.

    ```python
    context, alpha = self.attention(img_features, h)
    ```

4. **Combining and Transforming for Output Prediction**: The output word probability is computed by combining these elements and applying the learned weight matrices. In the code, this operation is currently condensed into one `self.deep_output` layer transforming the hidden state $ \mathbf{h}_t $. In a more complex or literal implementation of the DO-RNN, you would expect to see multiple such layers, each followed by a non-linear activation function.

    ```python
    output = self.deep_output(self.dropout(h))
    ```

## Changes to the implementation

In an effort to make the implementation more closely align with the paper, I made the following changes to the original implementation.

### DO-RNN according to the paper

As we saw above, the paper describes the DO-RNN as having multiple layers, each followed by a non-linear activation function. The original implementation only has one layer, transforming the hidden state of the LSTM.

```python

```